#### Merging scraped LTC statistics with ODHF

[How Ontario is responding to Covid-19](https://www.ontario.ca/page/how-ontario-is-responding-covid-19)

**Authors:** KT

---

In [1]:
import pandas as pd

In [2]:
odhf = pd.read_csv('/Users/kt/Documents/UBC_MDS/capstone/data/ODHF/odhf_v1.csv', engine='python')

In [4]:
ltc = pd.read_csv('../data/merged_ltc.csv')

In [7]:
ltc.head()

,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status
0,0,Almonte Country Haven,Almonte,82,<5,26,13,Active
1,1,Altamont Care Community,Scarborough,159,72,46,60,Active
2,2,Anson Place Care Centre,Hagersville,61,28,23,29,Active
3,3,Avalon Retirement Centre,Orangeville,137,0,0,<5,Active
4,4,Belmont House,Toronto,140,0,0,<5,Active


In [8]:
odhf.head()

,index,facility_name,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city,province,source_format_str_address,CSDname,CSDuid,Pruid,latitude,longitude
0,1,advanced facial & nasal surgery centre,active acute hospital,Hospitals,Canadian Institute for Health Information,NaN,NaN,NaN,T5M4G5,edmonton,ab,NaN,Edmonton,NaN,48,NaN,NaN
1,2,agecare � beverly centre glenmore,long term care,Nursing and residential care facilities,Canadian Institute for Health Information,NaN,NaN,NaN,T2V4S1,calgary,ab,NaN,Calgary,NaN,48,NaN,NaN
2,3,agecare � beverly centre lake midnapore,long term care,Nursing and residential care facilities,Canadian Institute for Health Information,NaN,NaN,NaN,T2X3S3,calgary,ab,NaN,Calgary,NaN,48,NaN,NaN
3,4,agecare � sagewood seniors community inc,long term care,Nursing and residential care facilities,Canadian Institute for Health Information,NaN,NaN,NaN,T1P0E2,strathmore,ab,NaN,Strathmore,NaN,48,NaN,NaN
4,5,agecare � seton,long term care,Nursing and residential care facilities,Canadian Institute for Health Information,NaN,NaN,NaN,T3M2M3,calgary,ab,NaN,Calgary,NaN,48,NaN,NaN


#### To Fix:
- remove symbols in odhf `facility_name`
- convert ltc `LTC Home` to lower case


In [31]:
import unicodedata

odhf['facility_name_clean'] = odhf['facility_name'].apply(lambda val: unicodedata.normalize('NFKD', val).encode('ascii', 'ignore').decode())

In [32]:
ltc['facility_name_clean'] = ltc['LTC Home'].map(lambda x: str(x).lower())

In [41]:
merged = pd.merge(odhf, ltc, how = 'inner', on = 'facility_name_clean')

In [40]:
len(odhf)

9039

In [38]:
len(ltc)

231

### Merge Summary

In [44]:
merged.groupby('source_facility_type')['facility_name_clean'].nunique()

source_facility_type
chronic care                     1
community support service        3
long-term care home            217
retirement home                  4
senior active living centre      1
Name: facility_name_clean, dtype: int64

#### Check other facility types to ensure merging by name is sensical

In [49]:
merged.loc[merged['source_facility_type']=='chronic care']

,index,facility_name_x,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city,...,facility_name_clean,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,facility_name_y
145,4842,providence healthcare,chronic care,Nursing and residential care facilities,Public Health Agency of Canada,NaN,3276,st clair ave e,M1L1W1,toronto,...,providence healthcare,33,Providence Healthcare,Scarborough,288,NaN,<5,NaN,inactive,providence healthcare


In [50]:
merged.loc[merged['source_facility_type']=='community support service']

,index,facility_name_x,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city,...,facility_name_clean,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,facility_name_y
11,2675,blue water rest home,community support service,Ambulatory health care services,Province of Ontario,NaN,37792,zurich-hensall road,N0M2T0,zurich,...,blue water rest home,8,Blue Water Rest Home,Zurich,65,<5,0,0,Active,blue water rest home
175,5440,st. joseph's lifecare centre,community support service,Ambulatory health care services,Province of Ontario,NaN,99,wayne gretzky parkway,N3S6T6,brantford,...,st. joseph's lifecare centre,134,St. Joseph's Lifecare Centre,Brantford,205,0,0,<5,Active,st. joseph's lifecare centre
224,6109,yee hong centre - markham,community support service,Ambulatory health care services,Province of Ontario,NaN,2780,bur oak avenue,L6B1C9,markham,...,yee hong centre - markham,55,Yee Hong Centre - Markham,Markham,200,NaN,0,NaN,inactive,yee hong centre - markham


In [52]:
merged.loc[merged['source_facility_type']=='senior active living centre']

,index,facility_name_x,source_facility_type,odhf_facility_type,provider,unit,street_no,street_name,postal_code,city,...,facility_name_clean,Unnamed: 0,LTC Home,City,Beds,Confirmed Resident Cases,Resident Deaths,Confirmed Staff Cases,Status,facility_name_y
156,5111,saint luke's place,senior active living centre,Nursing and residential care facilities,Province of Ontario,NaN,1624,franklin boulevard,N3C3P4,cambridge,...,saint luke's place,38,Saint Luke's Place,Cambridge,114,NaN,0,NaN,inactive,saint luke's place
